# EXIF 확인용 코드

In [1]:
from PIL import Image, ExifTags
import json, sys

def json_default(o):
    """Pillow IFDRational, bytes 등 JSON 직렬화 가능하게 변환"""
    try:
        from PIL.TiffImagePlugin import IFDRational
        if isinstance(o, IFDRational):
            return float(o)
    except Exception:
        pass
    if isinstance(o, bytes):
        try:
            return o.decode("utf-8", "ignore")
        except Exception:
            return o.hex()
    # 마지막 수단: 문자열화
    return str(o)

def _to_float(x):
    """IFDRational/분수 튜플 → float"""
    try:
        return float(x)
    except Exception:
        if isinstance(x, tuple) and len(x) == 2:
            a, b = x
            return float(a) / float(b)
        return float(x)

def dms_to_deg(dms):
    """(deg, min, sec) → 십진수"""
    d, m, s = (_to_float(dms[0]), _to_float(dms[1]), _to_float(dms[2]))
    return d + m/60.0 + s/3600.0

def summarize_exif(tag_map):
    """자주 쓰는 값만 간략 요약"""
    dt = tag_map.get("DateTimeOriginal") or tag_map.get("DateTime")
    gps_info = tag_map.get("GPSInfo")
    lat = lon = None
    if gps_info:
        gpstags = {ExifTags.GPSTAGS.get(k, k): v for k, v in gps_info.items()}
        try:
            lat = dms_to_deg(gpstags["GPSLatitude"])
            if gpstags.get("GPSLatitudeRef", "N") != "N":
                lat = -lat
            lon = dms_to_deg(gpstags["GPSLongitude"])
            if gpstags.get("GPSLongitudeRef", "E") != "E":
                lon = -lon
        except Exception:
            pass
    return {
        "datetime": dt,
        "gps": {"lat": lat, "lon": lon} if (lat is not None and lon is not None) else None,
    }



img = Image.open('bom.jpeg')
exif = img._getexif() or {}
# 키를 사람이 읽을 수 있는 이름으로 변환
tag_map = {ExifTags.TAGS.get(k, k): v for k, v in exif.items()}

# 1) 전체 EXIF (안전 직렬화)
print("=== FULL EXIF (sanitized) ===", file=sys.stderr)
print(json.dumps(tag_map, ensure_ascii=False, indent=2, default=json_default))

# 2) 요약본 (촬영시각 + GPS 십진수)
print("\n=== SUMMARY ===", file=sys.stderr)
print(json.dumps(summarize_exif(tag_map), ensure_ascii=False, indent=2))

{
  "GPSInfo": {
    "1": "N",
    "2": [
      34.0,
      44.0,
      28.22
    ],
    "3": "E",
    "4": [
      127.0,
      45.0,
      18.92
    ],
    "5": "\u0000",
    "6": 4.576926464112725,
    "7": [
      4.0,
      41.0,
      59.0
    ],
    "12": "K",
    "13": 0.018981222050081858,
    "16": "T",
    "17": 17.204895011368198,
    "23": "T",
    "24": 17.204895011368198,
    "29": "2025:08:09",
    "31": 4.748651528267963
  },
  "ResolutionUnit": 2,
  "ExifOffset": 234,
  "Make": "Apple",
  "Model": "iPhone 16 Pro",
  "Software": "18.5",
  "Orientation": 6,
  "DateTime": "2025:08:09 13:41:59",
  "YCbCrPositioning": 1,
  "XResolution": 72.0,
  "YResolution": 72.0,
  "HostComputer": "iPhone 16 Pro",
  "ExifVersion": "0232",
  "ComponentsConfiguration": "\u0001\u0002\u0003\u0000",
  "ShutterSpeedValue": 6.92349016059715,
  "DateTimeOriginal": "2025:08:09 13:41:59",
  "DateTimeDigitized": "2025:08:09 13:41:59",
  "ApertureValue": 1.6637544366004915,
  "BrightnessValue": 4.7

=== FULL EXIF (sanitized) ===

=== SUMMARY ===


In [2]:
# Jupyter 샘플: EXIF + GPS + (옵션) 날씨 조회
from PIL import Image, ExifTags
import requests, json, datetime as dt

# ---------- JSON/EXIF 유틸 ----------
def json_default(o):
    # Pillow의 IFDRational/bytes 직렬화 처리
    try:
        from PIL.TiffImagePlugin import IFDRational
        if isinstance(o, IFDRational):
            return float(o)
    except Exception:
        pass
    if isinstance(o, bytes):
        try:
            return o.decode("utf-8", "ignore")
        except Exception:
            return o.hex()
    return str(o)

def _to_float(x):
    try:
        from PIL.TiffImagePlugin import IFDRational
    except Exception:
        class IFDRational: ...
    if isinstance(x, (int, float)):
        return float(x)
    try:
        if isinstance(x, IFDRational):
            return float(x)
    except Exception:
        pass
    if isinstance(x, tuple) and len(x) == 2:
        a, b = x
        return float(a) / float(b)
    return float(x)

def dms_to_deg(dms):
    if isinstance(dms, (int, float)):
        return float(dms)
    d, m, s = (_to_float(dms[0]), _to_float(dms[1]), _to_float(dms[2]))
    return d + m/60.0 + s/3600.0

def parse_exif(path):
    with Image.open(path) as img:
        exif = img._getexif() or {}
    tags = {ExifTags.TAGS.get(k, k): v for k, v in exif.items()}
    # 촬영시각
    dt_str = tags.get("DateTimeOriginal") or tags.get("DateTime")
    # GPS
    gps_raw = tags.get("GPSInfo")
    gps = None
    if isinstance(gps_raw, dict):
        gpstags = {ExifTags.GPSTAGS.get(k, k): v for k, v in gps_raw.items()}
        lat = gpstags.get("GPSLatitude")
        lon = gpstags.get("GPSLongitude")
        if lat is not None and lon is not None:
            try:
                latv = dms_to_deg(lat)
                lonv = dms_to_deg(lon)
                lat_ref = str(gpstags.get("GPSLatitudeRef", "N")).upper()
                lon_ref = str(gpstags.get("GPSLongitudeRef", "E")).upper()
                latv = abs(latv) if lat_ref == "N" else -abs(latv)
                lonv = abs(lonv) if lon_ref == "E" else -abs(lonv)
                gps = {"lat": round(latv, 7), "lon": round(lonv, 7)}
            except Exception:
                gps = None
    return tags, dt_str, gps

def date_from_exif(dt_str: str | None) -> str:
    # 'YYYY:MM:DD HH:MM:SS' → 'YYYY-MM-DD' (없으면 오늘)
    if not dt_str:
        return dt.date.today().isoformat()
    try:
        t = dt.datetime.strptime(dt_str, "%Y:%m:%d %H:%M:%S")
        return t.date().isoformat()
    except Exception:
        return dt_str[:10].replace(":", "-")

def fetch_weather(lat: float, lon: float, iso_date: str):
    try:
        params = {
            "latitude": lat,
            "longitude": lon,
            "daily": ["temperature_2m_max","temperature_2m_min","precipitation_sum","weathercode"],
            "timezone": "auto",
            "start_date": iso_date,
            "end_date": iso_date,
        }
        r = requests.get("https://api.open-meteo.com/v1/forecast", params=params, timeout=10)
        if not r.ok:
            return None
        j = r.json()
        d = j.get("daily") or {}
        return {
            "date": (d.get("time") or [iso_date])[0],
            "tmax": (d.get("temperature_2m_max") or [None])[0],
            "tmin": (d.get("temperature_2m_min") or [None])[0],
            "precip": (d.get("precipitation_sum") or [None])[0],
            "code": (d.get("weathercode") or [None])[0],
        }
    except Exception:
        return None

# ---------- 여기만 바꿔서 실행 ----------
IMAGE = "bom.jpeg"   # 테스트할 이미지 경로
DO_WEATHER = True      # 날씨도 조회할지

# ---------- 실행 ----------
tags, dt_str, gps = parse_exif(IMAGE)
iso_date = date_from_exif(dt_str)

print("=== FULL EXIF (sanitized) ===")
print(json.dumps(tags, ensure_ascii=False, indent=2, default=json_default))

out = {"datetime": dt_str, "date": iso_date, "gps": gps, "weather": None}
if DO_WEATHER and gps:
    out["weather"] = fetch_weather(gps["lat"], gps["lon"], iso_date)

print("\n=== SUMMARY ===")
print(json.dumps(out, ensure_ascii=False, indent=2))

if not gps:
    print("[note] EXIF에 GPS가 없어 날씨를 조회할 수 없습니다.")
elif DO_WEATHER and not out["weather"]:
    print("[note] 날씨 API 조회 실패 또는 해당 날짜 데이터 없음.")


=== FULL EXIF (sanitized) ===
{
  "GPSInfo": {
    "1": "N",
    "2": [
      34.0,
      44.0,
      28.22
    ],
    "3": "E",
    "4": [
      127.0,
      45.0,
      18.92
    ],
    "5": "\u0000",
    "6": 4.576926464112725,
    "7": [
      4.0,
      41.0,
      59.0
    ],
    "12": "K",
    "13": 0.018981222050081858,
    "16": "T",
    "17": 17.204895011368198,
    "23": "T",
    "24": 17.204895011368198,
    "29": "2025:08:09",
    "31": 4.748651528267963
  },
  "ResolutionUnit": 2,
  "ExifOffset": 234,
  "Make": "Apple",
  "Model": "iPhone 16 Pro",
  "Software": "18.5",
  "Orientation": 6,
  "DateTime": "2025:08:09 13:41:59",
  "YCbCrPositioning": 1,
  "XResolution": 72.0,
  "YResolution": 72.0,
  "HostComputer": "iPhone 16 Pro",
  "ExifVersion": "0232",
  "ComponentsConfiguration": "\u0001\u0002\u0003\u0000",
  "ShutterSpeedValue": 6.92349016059715,
  "DateTimeOriginal": "2025:08:09 13:41:59",
  "DateTimeDigitized": "2025:08:09 13:41:59",
  "ApertureValue": 1.663754436600


=== SUMMARY ===
{
  "datetime": "2025:08:09 13:41:59",
  "date": "2025-08-09",
  "gps": {
    "lat": 34.7411722,
    "lon": 127.7552556
  },
  "weather": {
    "date": "2025-08-09",
    "tmax": 26.0,
    "tmin": 23.5,
    "precip": 60.3,
    "code": 65
  }
}


In [3]:
# Jupyter 셀: 이미지 → EXIF(시간/GPS) → 주소(역지오코딩) → 날씨(해당 날짜)
from PIL import Image, ExifTags
import requests, json, datetime as dt, time

# ===== 설정 (여기만 바꿔서 실행) =====
IMAGE_PATH   = "bom.jpeg"  # 테스트할 이미지 경로
DO_ADDRESS   = True          # True면 역지오코딩 수행
DO_WEATHER   = True          # True면 날씨 조회 (GPS 필요)
LANG         = "ko"          # 주소 언어 우선순위
NOMINATIM_UA = "dayline-mcp/1.0 (contact: you@example.com)"  # ← 연락처/도메인으로 바꿔주세요

# ===== JSON/EXIF 유틸 =====
def json_default(o):
    # Pillow의 IFDRational/bytes 직렬화 처리
    try:
        from PIL.TiffImagePlugin import IFDRational
        if isinstance(o, IFDRational):
            return float(o)
    except Exception:
        pass
    if isinstance(o, bytes):
        try:
            return o.decode("utf-8", "ignore")
        except Exception:
            return o.hex()
    return str(o)

def _to_float(x):
    try:
        from PIL.TiffImagePlugin import IFDRational
        if isinstance(x, IFDRational):
            return float(x)
    except Exception:
        pass
    if isinstance(x, (int, float)):
        return float(x)
    if isinstance(x, tuple) and len(x) == 2:  # (num, den)
        a, b = x
        return float(a) / float(b)
    return float(x)

def dms_to_deg(dms):
    # (deg, min, sec) → 십진수; 이미 숫자면 그대로
    if isinstance(dms, (int, float)):
        return float(dms)
    d, m, s = (_to_float(dms[0]), _to_float(dms[1]), _to_float(dms[2]))
    return d + m/60.0 + s/3600.0

def parse_exif(path):
    with Image.open(path) as img:
        exif = img._getexif() or {}
    tags = {ExifTags.TAGS.get(k, k): v for k, v in exif.items()}

    # 촬영시각
    dt_str = tags.get("DateTimeOriginal") or tags.get("DateTime")

    # GPS
    gps_raw = tags.get("GPSInfo")
    gps = None
    if isinstance(gps_raw, dict):
        gpstags = {ExifTags.GPSTAGS.get(k, k): v for k, v in gps_raw.items()}
        lat = gpstags.get("GPSLatitude")
        lon = gpstags.get("GPSLongitude")
        if lat is not None and lon is not None:
            try:
                latv = dms_to_deg(lat)
                lonv = dms_to_deg(lon)
                lat_ref = str(gpstags.get("GPSLatitudeRef", "N")).upper()
                lon_ref = str(gpstags.get("GPSLongitudeRef", "E")).upper()
                latv =  abs(latv) if lat_ref == "N" else -abs(latv)
                lonv =  abs(lonv) if lon_ref == "E" else -abs(lonv)
                gps = {"lat": round(latv, 7), "lon": round(lonv, 7)}
            except Exception:
                gps = None
    return tags, dt_str, gps

def exif_date_to_iso(dt_str: str | None) -> str:
    # 'YYYY:MM:DD HH:MM:SS' → 'YYYY-MM-DD' (없으면 오늘)
    if not dt_str:
        return dt.date.today().isoformat()
    try:
        t = dt.datetime.strptime(dt_str, "%Y:%m:%d %H:%M:%S")
        return t.date().isoformat()
    except Exception:
        return dt_str[:10].replace(":", "-")

# ===== 역지오코딩(Nominatim) =====
def reverse_geocode(lat: float, lon: float, lang: str = "ko", user_agent: str | None = None):
    try:
        headers = {"User-Agent": user_agent or "dayline-mcp/1.0"}
        params = {
            "format": "jsonv2",
            "lat": lat,
            "lon": lon,
            "zoom": 14,               # 동/동네 수준
            "addressdetails": 1,
            "accept-language": lang,
        }
        r = requests.get("https://nominatim.openstreetmap.org/reverse",
                         params=params, headers=headers, timeout=10)
        if not r.ok:
            return None
        j = r.json()
        addr = j.get("address") or {}
        city = addr.get("city") or addr.get("town") or addr.get("village") or addr.get("county")
        district = addr.get("district") or addr.get("borough") or addr.get("suburb")
        hood = addr.get("neighbourhood") or addr.get("quarter")
        parts = [p for p in [city, district, hood] if p]
        if parts:
            return " ".join(dict.fromkeys(parts))
        return j.get("display_name")
    except Exception:
        return None

# ===== Open-Meteo 날씨 =====
def fetch_weather(lat: float, lon: float, iso_date: str):
    try:
        params = {
            "latitude": lat,
            "longitude": lon,
            "daily": ["temperature_2m_max","temperature_2m_min","precipitation_sum","weathercode"],
            "timezone": "auto",
            "start_date": iso_date,
            "end_date": iso_date,
        }
        r = requests.get("https://api.open-meteo.com/v1/forecast", params=params, timeout=10)
        if not r.ok:
            return None
        j = r.json()
        d = j.get("daily") or {}
        return {
            "date": (d.get("time") or [iso_date])[0],
            "tmax": (d.get("temperature_2m_max") or [None])[0],
            "tmin": (d.get("temperature_2m_min") or [None])[0],
            "precip": (d.get("precipitation_sum") or [None])[0],
            "code": (d.get("weathercode") or [None])[0],
        }
    except Exception:
        return None

# ===== 실행 =====
tags, dt_str, gps = parse_exif(IMAGE_PATH)
iso_date = exif_date_to_iso(dt_str)

print("=== FULL EXIF (sanitized) ===")
print(json.dumps(tags, ensure_ascii=False, indent=2, default=json_default))

out = {"datetime": dt_str, "date": iso_date, "gps": gps, "address": None, "weather": None}

# 주소
if DO_ADDRESS and gps:
    out["address"] = reverse_geocode(gps["lat"], gps["lon"], lang=LANG, user_agent=NOMINATIM_UA)
    # 예의상 과도 호출 방지 (여러 장 루프에서 쓸 때)
    time.sleep(1.0)

# 날씨
if DO_WEATHER and gps:
    out["weather"] = fetch_weather(gps["lat"], gps["lon"], iso_date)

print("\n=== SUMMARY ===")
print(json.dumps(out, ensure_ascii=False, indent=2))

if not gps:
    print("[note] EXIF에 GPS가 없어 주소/날씨를 조회할 수 없습니다.")
elif DO_WEATHER and not out["weather"]:
    print("[note] 날씨 API 조회 실패 또는 해당 날짜 데이터 없음.")


=== FULL EXIF (sanitized) ===
{
  "GPSInfo": {
    "1": "N",
    "2": [
      34.0,
      44.0,
      28.22
    ],
    "3": "E",
    "4": [
      127.0,
      45.0,
      18.92
    ],
    "5": "\u0000",
    "6": 4.576926464112725,
    "7": [
      4.0,
      41.0,
      59.0
    ],
    "12": "K",
    "13": 0.018981222050081858,
    "16": "T",
    "17": 17.204895011368198,
    "23": "T",
    "24": 17.204895011368198,
    "29": "2025:08:09",
    "31": 4.748651528267963
  },
  "ResolutionUnit": 2,
  "ExifOffset": 234,
  "Make": "Apple",
  "Model": "iPhone 16 Pro",
  "Software": "18.5",
  "Orientation": 6,
  "DateTime": "2025:08:09 13:41:59",
  "YCbCrPositioning": 1,
  "XResolution": 72.0,
  "YResolution": 72.0,
  "HostComputer": "iPhone 16 Pro",
  "ExifVersion": "0232",
  "ComponentsConfiguration": "\u0001\u0002\u0003\u0000",
  "ShutterSpeedValue": 6.92349016059715,
  "DateTimeOriginal": "2025:08:09 13:41:59",
  "DateTimeDigitized": "2025:08:09 13:41:59",
  "ApertureValue": 1.663754436600


=== SUMMARY ===
{
  "datetime": "2025:08:09 13:41:59",
  "date": "2025-08-09",
  "gps": {
    "lat": 34.7411722,
    "lon": 127.7552556
  },
  "address": "여수시 수정동",
  "weather": {
    "date": "2025-08-09",
    "tmax": 26.0,
    "tmin": 23.5,
    "precip": 60.3,
    "code": 65
  }
}
